## 롯데손보등 분급 수수료 정산 프로그램 : 본 프로그램 사용시 파일명,회사명 확인 후 실행바람
  1. 분급,선급원장 읽기 : 첫행은 반드시 컬럼명이어야  
  2. 분급원장 전처리 : kj계약 추출, 문홍주건 정제, 지점코드 세팅  
  3. 선급원장 정체: 롯데,장기  
  4. 증번,회차별 sort  
  5. 선급,분급 차이 산출  
  6. 요약,목록 시트 출력  
 
 본 스크립트는 분급수수료정산.py로 제작되어 실행파일화 함: d:/글로벌/분급정산  

In [131]:
# import sys
# sys.path.insert(0, 'e:/python/mypkgs')
import pandas as pd
# import mypkgs.GladUtil as glad
from python.mypkgs import GladUtil  as glad


inFile1='E:/글로벌백업(180329)/04.수수료작업/분급이자/202004_롯데 분급 원장.xlsx'  #분급원장
inFile2='E:/글로벌백업(180329)/04.수수료작업/202004수수료/2020-04 수입원장(KJ).xlsx'       #선급원장
outFile='E:/글로벌백업(180329)/04.수수료작업/202004수수료/2020-04 롯데정산결과(KJ).xlsx'   #출력파일
company='롯데손보'
#---------------------------------------------
df=pd.read_excel(inFile1, header=0)
df=df[df.본부=='MPKJ']
df['계약일자']=df.계약일자.map(lambda x: str(x)[:8])
df=df[~((df.사원명=='문홍주') & (df.계약일자 < '20180601'))]  #문홍주 구소속건 삭제
df=glad.BrhConv(df,'지점그룹')                               #메트로지점분리, 직영/지사구분

In [122]:
dfw=pd.read_excel(inFile2, header=0, sheet_name='전체')  

In [123]:
#선급 원장: 기존 방식의 계산값(글로벌 -> MP)
df2=dfw[(dfw.보험사==company)&(dfw.계약종류=='장기')]
df2.rename(columns={'증권번호':'분급증번'}, inplace=True)

C:\Users\user\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [124]:
#concat를 하기 위해 index순번을 재부여
df=df.sort_values(by=['증권번호','납입회차','상태','수수료']).reset_index()
df2=df2.sort_values(by=['분급증번','납입회차','상태','수수료계']).reset_index()
df=pd.concat([df, df2.수수료계,df2.분급증번], axis=1)  #차례대로 합침
df.rename(columns={'수수료계':'선급수수료','수수료':'분급수수료'}, inplace=True)
df['차액(선급-분급)']=df.선급수수료 - df.분급수수료

In [125]:
#요약시트 생성
dfT=pd.DataFrame(df.groupby('지점')['선급수수료','분급수수료','차액(선급-분급)'].agg('sum'))
직영=['엠피레전드','엠피여주','엠피의정부','제이엔제이','씨티엠지점']
dfT.loc[dfT.index.isin(직영),'%']=100 
dfT.loc[~dfT.index.isin(직영),'%']=70
dfT['kj']=dfT['%'] * dfT['차액(선급-분급)'] / 100  #귀속분 산출
s1=dfT.sum()    #요약값 산출
s1['%']=0       
dfT.loc['합계']=s1        #마지막 row에 합계추가
dfT['이자']= (dfT['kj'] * 0.03 / 12).astype(int)  #마지막 column열에 이자 3% 적용

with pd.ExcelWriter(outFile) as writer:  
    dfT.to_excel(writer, sheet_name='요약',index=True)
    df.to_excel(writer, sheet_name='목록',index=False) 

#del dfT,s1


C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [126]:
if  (df.선급수수료.sum() - df2.수수료계.sum()) != 0:
     print('오류발생xxxxxxxxxxxxxxxxxxxxxxxxxx')
elif  df.선급수수료.sum() != s1.선급수수료:        
     print('오류발생-분급누락건 점검요~ xxxxxxx')
        ##분급원장에 없는 증번은 수입원장에서 분급원장으로 건을 복사 후 실행요망
else: print('정상종료-------------------------')

print('선급수수료-matched = {:,.0f}'.format(s1.선급수수료) )
print('선급수수료-원장 = {:,.0f}'.format(df2.수수료계.sum() ))
print('분급수수료-원장 = {:,.0f}'.format(df.분급수수료.sum() ) )
print('선급-분급 차액 = {:,.0f}'.format(df['차액(선급-분급)'].sum() ) )


정상종료-------------------------
선급수수료-matched = 35,178,813
선급수수료-원장 = 35,178,813
분급수수료-원장 = 36,759,938
선급-분급 차액 = -1,581,125


In [ ]:
temp=df[df.증권번호==df2.증권번호]['NO'].count()

###아래는 오류 났을때 사용할 임시 로직  

In [ ]:
df_kj=df.groupby('증권번호')['지급','환수'].sum().sort_values('증권번호')

In [ ]:
df2_kj=pd.DataFrame(df2.groupby('증권번호')['수수료계'].sum()).sort_values('증권번호')

In [ ]:
# 선급원장금액 생성
df_kj['원장']=df2_kj.수수료계

In [ ]:
#df[df.증권번호.isin(df3_kj.index)]
df_T=df2[['증권번호','납입회차','수수료계']]
df=df.join(df_T.set_index(['증권번호','납입회차'])['수수료계'], on=['증권번호','납입회차'])

In [ ]:
df3_kj=df_kj[df_kj.지급+df_kj.환수 != df_kj.원장]

In [ ]:
pd.DataFrame([df.선급수수료.sum(),df.분급수수료.sum(),df['차액(선급-분급)'].sum()],['선급','분급','차액'],columns=['금액'])

In [132]:
df.columns

Index(['NO', '지급년월', '적용일', '영수구분', '보험사', '계약종류', '증권번호', 'FC코드', '총괄', '본부',
       '지점', '직할지점', '팀', '사원번호', '사원명', '납입회차', '상태', '상품명', '수수료', '보험료',
       '원수사성적', '계약일자', '영수일자', '자동차_물건구분', '자동차_보험종목', '지급', '환수', '위촉(Y/N)',
       'Unnamed: 28', 'Unnamed: 29'],
      dtype='object')

In [87]:
df.loc[df.증권번호!=df.분급증번][['증권번호','분급증번']]
# df[['증권번호','분급증번']]

,증권번호,분급증번
2007,LA20192119663000,LA20192117746000
2008,LA20192119875000,LA20192119663000
2009,LA20192121340000,LA20192119875000
2010,LA20192121544000,LA20192121340000
2011,LA20192121966000,LA20192121544000
...,...,...
2397,LA20202540661000,LA20202535910000
2398,NaN,LA20202535917000
2399,NaN,LA20202538853000
2400,NaN,LA20202540619000
